<a href="https://colab.research.google.com/github/yc-chen10/113-1-finance/blob/main/week13%E4%BD%9C%E6%A5%AD_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 發送 HTTP 請求以獲取網路上的資料
import requests

# 用來處理和分析結構化數據
import pandas as pd

# 用來處理時間
import datetime as dt

import time

import numpy as np
import random

####抓股票資料

2330
2454
2303
3711
3034
2379


In [ ]:
# "一詮"股票代號
stock_id = '2486'

# 獲取今天的日期（型別：str）
date = dt.date.today().strftime("%Y%m%d")
# date = yyyymmdd

print('今日日期：', date)

# 取得證交所網站資料，帶入網址參數 date：時間，stock_id：股票代號
# https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240923&stockNo=2486
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')

# 將結果轉換 JSON 格式
json_data = stock_data.json()

# 將 JSON 格式轉換爲 Dataframe 格式，
# 方便在 Python 中處理結構化資料
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])

df.head()

今日日期： 20241210


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/12/02,"4,380,182","484,081,055",111.00,112.00,109.00,110.50,+1.00,"2,642"
1,113/12/03,"20,118,123","2,388,317,316",112.00,121.50,112.00,120.50,+10.00,"11,597"
2,113/12/04,"20,279,814","2,424,225,272",121.00,123.00,117.00,118.50,-2.00,"16,735"
3,113/12/05,"24,214,277","2,969,634,843",120.00,124.50,119.00,122.50,+4.00,"14,458"
4,113/12/06,"11,255,798","1,342,014,582",122.00,122.50,117.50,117.50,-5.00,"6,990"


#### def function

In [ ]:
def get_stock_data(stock_no, year, month):
    # 將日期格式化為 'yyyymmdd'，並指定為該月份的第一天
    date = f"{year}{str(month).zfill(2)}01"
    url = f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={date}&stockNo={stock_no}"

    # 發送HTTP請求並獲取響應
    response = requests.get(url)
    #休息保護 避免被當成攻擊
    time.sleep(1)
    if response.status_code == 200:
        try:
            # 將結果轉換為JSON
            data = response.json()
            if data['stat'] == 'OK':
                # 返回數據中的每日交易資料
                return pd.DataFrame(data['data'], columns=data['fields'])
            else:
                print(f"No data for {year}-{month}, stat: {data.get('stat', 'N/A')}")
                return None
        except requests.exceptions.JSONDecodeError:
            print(f"Invalid JSON response for {year}-{month}. Response content: {response.content}")
            return None
    else:
        print(f"Failed to fetch data for {year}-{month}")
        return None


In [ ]:
def get_multiple_months(stock_no, start_year, start_month, end_year, end_month):
    all_data = []

    # 循環通過年份和月份來調用API
    current_year = start_year
    current_month = start_month

    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
        # 獲取單月的股票資料
        monthly_data = get_stock_data(stock_no, current_year, current_month)
        if monthly_data is not None:
            all_data.append(monthly_data)

        # 更新到下一個月份
        if current_month == 12:
            current_month = 1
            current_year += 1
        else:
            current_month += 1

    # 合併所有月份的數據
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return None


#### 抓資料 main程式


In [ ]:
# 股票代號列表
stock_ids = ['2330', '2454', '2303', '3711', '3034', '2379']
start_year, start_month, end_year, end_month = 2023, 11, 2024, 11

# 取得所有股票的歷史收盤價資料
stock_data = {}
for stock_no in stock_ids:
    stock_data[stock_no] = get_multiple_months(stock_no, start_year, start_month, end_year, end_month)

In [ ]:
stock_data

{'2330':             日期        成交股數            成交金額       開盤價       最高價       最低價  \
 0    112/11/01  14,028,881   7,432,561,094    533.00    533.00    527.00   
 1    112/11/02  31,075,778  16,846,430,251    536.00    547.00    535.00   
 2    112/11/03  16,673,456   9,129,495,705    547.00    549.00    545.00   
 3    112/11/06  31,297,186  17,321,453,879    553.00    556.00    550.00   
 4    112/11/07  16,990,175   9,406,486,499    552.00    555.00    550.00   
 ..         ...         ...             ...       ...       ...       ...   
 258  113/11/25  54,443,912  56,293,841,760  1,045.00  1,050.00  1,030.00   
 259  113/11/26  31,309,332  31,750,783,325  1,015.00  1,020.00  1,010.00   
 260  113/11/27  34,602,143  34,822,499,480  1,010.00  1,020.00  1,000.00   
 261  113/11/28  37,166,049  37,181,706,464  1,005.00  1,005.00    992.00   
 262  113/11/29  43,621,329  43,637,043,385    995.00  1,010.00    994.00   
 
           收盤價    漲跌價差     成交筆數  
 0      528.00   -1.00   15,452 

In [ ]:
for stock_key, data in stock_data.items():
    if data is not None:  # 檢查資料是否為有效 DataFrame
        print(f"Stock {stock_key}:")
        #print(data.head())  # 檢查每支股票的前幾筆資料
    else:
        print(f"Stock {stock_key} contains no data.")


Stock 2330:
Stock 2454:
Stock 2303:
Stock 3711:
Stock 3034:
Stock 2379:


In [ ]:
original_data = stock_data.copy()

#### 時間格式轉換def

In [ ]:
def convert_minguo_to_ad(date_str):
    # 113/09/10 透過「/」分割成清單 [113, 09, 10]
    # 分配給變數 year=113, month=09, day=10
    year, month, day = date_str.split('/')
    ad_year = int(year) + 1911  # 將民國年轉換為西元年
    return f"{ad_year}/{month}/{day}"

In [ ]:
for stock_key in stock_data:
    # 所有民國日期都轉西元日期 e.g.113/09/10 -> 2024/09/10
    stock_data[stock_key]['日期'] = stock_data[stock_key]['日期'].apply(convert_minguo_to_ad)
    # 轉換為 pandas 中的 datetime 格式
    stock_data[stock_key]['日期'] = pd.to_datetime(stock_data[stock_key]['日期'], format='%Y/%m/%d')

In [ ]:
stock_data['2330'].head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2023-11-01,"14,028,881","7,432,561,094",533.00,533.00,527.00,528.00,-1.00,"15,452"
1,2023-11-02,"31,075,778","16,846,430,251",536.00,547.00,535.00,547.00,+19.00,"32,075"
2,2023-11-03,"16,673,456","9,129,495,705",547.00,549.00,545.00,549.00,+2.00,"17,911"
3,2023-11-06,"31,297,186","17,321,453,879",553.00,556.00,550.00,550.00,+1.00,"32,684"
4,2023-11-07,"16,990,175","9,406,486,499",552.00,555.00,550.00,555.00,+5.00,"15,356"


#### 轉數字 numeric

In [ ]:
for stock_key in stock_data:
    stock_data[stock_key]['收盤價'] = stock_data[stock_key]['收盤價'].apply(pd.to_numeric, errors='coerce')
    stock_data[stock_key]['開盤價'] = stock_data[stock_key]['開盤價'].apply(pd.to_numeric, errors='coerce')
    stock_data[stock_key]['最高價'] = stock_data[stock_key]['最高價'].apply(pd.to_numeric, errors='coerce')
    stock_data[stock_key]['最低價'] = stock_data[stock_key]['最低價'].apply(pd.to_numeric, errors='coerce')

In [ ]:
stock_data['2330'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      263 non-null    datetime64[ns]
 1   成交股數    263 non-null    object        
 2   成交金額    263 non-null    object        
 3   開盤價     215 non-null    float64       
 4   最高價     211 non-null    float64       
 5   最低價     219 non-null    float64       
 6   收盤價     212 non-null    float64       
 7   漲跌價差    263 non-null    object        
 8   成交筆數    263 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 18.6+ KB


#### 基因演算法

In [ ]:
# 計算每支股票的平均回報率和標準差
stock_metrics = {}
for stock_no, data in stock_data.items():
    data['回報率'] = data['收盤價'].pct_change()
    mean_return = data['回報率'].mean()
    risk = data['回報率'].std()
    stock_metrics[stock_no] = {'mean_return': mean_return, 'risk': risk}

<ipython-input-33-6fec9113f13e>:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['回報率'] = data['收盤價'].pct_change()
<ipython-input-33-6fec9113f13e>:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['回報率'] = data['收盤價'].pct_change()


In [ ]:


# 基因演算法設置
population_size = 20
num_generations = 50
mutation_rate = 0.1
chromosome_length = len(stock_ids)

In [ ]:
# 適應函數
def fitness_function(chromosome):
    selected_stocks = [stock_ids[i] for i, gene in enumerate(chromosome) if gene == 1]
    if not selected_stocks:
        return 0  # 避免全選 0
    portfolio_return = np.mean([stock_metrics[stock]['mean_return'] for stock in selected_stocks])
    portfolio_risk = np.mean([stock_metrics[stock]['risk'] for stock in selected_stocks])
    fitness = portfolio_return - 0.5 * portfolio_risk  # 風險厭惡係數
    return fitness

In [ ]:
# 初始化族群
population = [np.random.choice([0, 1], size=chromosome_length) for _ in range(population_size)]

In [ ]:



# 基因演算法主迴圈
for generation in range(num_generations):
    # 評估適應值
    fitness_values = [fitness_function(chromosome) for chromosome in population]

    # 選擇 (輪盤選擇)
    total_fitness = sum(fitness_values)
    probabilities = [f / total_fitness for f in fitness_values]
    selected_indices = np.random.choice(range(population_size), size=population_size, p=probabilities)
    new_population = [population[i] for i in selected_indices]

    # 交配
    next_generation = []
    for i in range(0, population_size, 2):
        parent1 = new_population[i]
        parent2 = new_population[(i + 1) % population_size]
        crossover_point = random.randint(1, chromosome_length - 1)
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        next_generation.extend([child1, child2])

    # 突變
    for i in range(population_size):
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, chromosome_length - 1)
            next_generation[i][mutation_point] = 1 - next_generation[i][mutation_point]

    population = next_generation


In [ ]:
# 最佳解
best_chromosome = max(population, key=fitness_function)
selected_stocks = [stock_ids[i] for i, gene in enumerate(best_chromosome) if gene == 1]

In [ ]:

# 輸出結果
print("最佳投資組合:")
print("選股:", selected_stocks)
print("預期回報率:", np.mean([stock_metrics[stock]['mean_return'] for stock in selected_stocks]))
print("預期風險:", np.mean([stock_metrics[stock]['risk'] for stock in selected_stocks]))

最佳投資組合:
選股: ['2330', '2303', '3034', '2379']
預期回報率: 0.0008870864755727547
預期風險: 0.01849865422538522
